<img src="REcoM.png" alt="Drawing" style="width: 900px; margin: 0 auto">
<p style="text-align: center;">This notebook is an extensive demonstration of the *py_recom* module. *py_recom* is a tool for the evaluation of REcoM model outputs.</p>

### Table of Contents

* [0. Configuration](#CONFIG)
            
* [4 Ocean Productivity & ecosystem](#PP)
     * [4.1 Maps](#PPMAPS)
         * [4.1.1 Glogal Ocean Chl a](#OCCCIPPMAP)
         * [4.1.2 Southern Ocean Chl a](#JOHNPPMAP)
         * [4.1.3 Arctic Ocean Chl a](#ARCCHLPMAP)
         * [4.1.4 Global Ocean NPP](#NPPGLOBAL)
         * [4.1.5 Arctic Ocean NPP](#NPPARCTICMAP)
         * [4.1.6 Maredat comparison](#MAREDAT)
         * [4.1.7 Limiting Factors](#LIMFACT)
     * [4.3 Latitudinal distributions](#LATPP)
     * [4.2 Time-series](#TSPP)
     * [4.4 Seasonal cycle](#SCyc)
         * [4.4.1 Chl-a](#SCycChla)
         * [4.4.2 NPP](#SCycNPP)

# 0. Configuration<a class="anchor" id="CONFIG"></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')
%autosave 5
# necessary modules -------------------------------------------------------------------------------------
import socket
import sys
import os
import io
import contextlib

home = os.path.expanduser("~/master_thesis")
sys.path.append(home+'/py_f2recom/GlobalAssessment/')
sys.path.append(home+ '/py_f2recom/modules/')
sys.path.append(home+ '/py_f2recom/modules/pyfesom2')
sys.path.append(home+ '/py_f2recom/modules/cmocean-master/')
sys.path.append(home+'/py_f2recom/modules/SkillMetrics/')

import pyfesom2 as pf
import skill_metrics as sm
import cmocean as cmo
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import time
from datetime import date
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from netCDF4 import Dataset

Autosaving every 5 seconds
osgeo is not installed, conversion to Geo formats like Geotiff (fesom2GeoFormat) will not work.


In [2]:
#    Only modify parameters and paths in this cell       #
#  In principle, you don't have to modify anything else  #


# run specification -------------------------------------------------------------------------------------   
#simu_name = 'projects/MarESys/mseifert/fesom/fesom2-recom/Sina/Sina_photodamage_ON' # only usefull if you want to save figures
simu_name = 'simuel001/blanchard_newparams_20240822'


# defines paths ----------------------------------------------------------------------------------------- 
if socket.gethostname()[:5] == 'blogi':
    meshpath = '/scratch/usr/hbkoziel/mesh/farc'
    resultpath = '/scratch/projects/hbk00083/model_outputs/fesom2.1_recom'+simu_name+'/'
    savepath = home+'/pyfesom2/codes/py_f2recom_develop/outputs/'+simu_name+'/'
    evalpath      = '/scratch/usr/hbkoziel/evaluation/'
    evalpath2      = '/scratch/usr/hbkoziel/corrected_input/'
elif socket.gethostname()[:5] in ['albed','prod-']:
    #resultpath = '/albedo/work/projects/p_bio/model_output/A_riv'
    #resultpath = '/albedo/work/projects/MarESys/GCB2022/A'
    #resultpath = '/albedo/work/'+simu_name+'/'                # for 3p model output
    resultpath = '/albedo/scratch/user/'+simu_name+'/'          # for 4p model output
    savepath = home+'/master_thesis/py_f2recom_4phy/'+simu_name+'/'
    evalpath = '/albedo/work/projects/p_pool_recom/eval/'
    meshpath = '/albedo/work/projects/p_bio/mesh/core2/'
    #meshpath = '/albedo/work/user/yye/fesom2/meshes/core2_albedo/'
else:
    print('sorry, machine unknown, please customize your paths yourself')

# period of analysis ------------------------------------------------------------------------------------
first_year_maps = 1958
first_year = 1974
last_year  = 1978
    
years = np.arange(first_year,last_year+1,1)
years_last10 = np.arange(first_year_maps,last_year+1,1)

# specification of analysis ------------------------------------------------------------------------------------
layerwise = False
depths = (0,50,200,1000,2000) # If layerwise is True, you can define depths here, by defaut: (0,50,200,1000,2000,4000)
uplow = [0, 100]
mapproj = 'rob'
# Map projection ptions are : Mercator (merc), Plate Carree (pc),
# North Polar Stereo (np), South Polar Stereo (sp),  Robinson (rob)
# Robinson projection is quite time consuming.

In [3]:
# export of analysis ------------------------------------------------------------------------------------
# Be aware that exporting figures may alter (crop) the display but the printed figures are okay
# This is because of bugs in the 'constrained_layout' matplotlib experimental function 
# that may be fixed in the future matplotlib version but out of our control
# If you prefer having a nice HTML, savefig must be turned off
#--------------------------------------------------------------------------------------------------------
today = date.today().strftime("_%Y_%m_%d")
savefig = True 
htmlname     =  simu_name+'_'+ today +'_eco.html'
htmlpath = savepath
verbose = True

if not os.path.exists(htmlpath): # create folders if do not exist
    os.makedirs(htmlpath)
if not os.path.exists(savepath):
    os.makedirs(savepath)
    
# initialization file specifications -----------------------------------------------------------

matfileChlGloOCCCI       = evalpath+'climatology_annual_chl_1deg_OCCCI_2012_2015.mat'
ncfileChlSouthernJohnson = evalpath+'Johnson2013_MEAN_1x1_Chl_mg_m3.npy'
matfileNPPvgpm           = evalpath+'VGPM_CLIM.mat'
matfileNPPcpbm           = evalpath+'CBPM_CLIM.mat'
ncfileMaredatDia         = evalpath+'MarEDat20120716Diatoms.nc'
ncfileMaredatCocco       = evalpath+'MarEDat20130523Coccolithophores.nc'
ncfileMaredatPhaeo       = evalpath+'MarEDat20120424Phaeocystis_filtered.nc'
ncfileMaredatMicro       = evalpath+'MarEDat20120424Microzooplankton.nc'
ncfileMaredatMeso        = evalpath+'MarEDat20120524Mesozooplankton.nc'
ncfileMaredatMacro       = evalpath+'MarEDat20120216Macrozooplankton.nc'
ncfileChlArcLewis        = evalpath+'AO_SAT/LEWIS_CLIMATOLOGY_2003_2021_CHL.nc'
ncfileChlArcOCCCI        = evalpath+'AO_SAT/OCCCI_CLIMATOLOGY_2000_2019_May_Sept_CHL.nc'
npfileNPPArcLewis        = evalpath+'NPP_ARRIGO_2003_2018_reg.npy'
npfileNPParcCMEMS        = evalpath+'NPP_CMEMS_2003_2018_reg.npy'

# visual check
if(verbose):
    print('Processing years {4} to {5}\n\nReading out of {0}\nStoring graphs to {1}\nStoring html to {2} as {3}'.format(
        resultpath, savepath, htmlpath, htmlname,years[0],years[-1]))
    print('\nLast ten years are \n{0}'.format(years_last10))
    
# mesh initialization -----------------------------------------------------------
mesh = pf.load_mesh(meshpath)
mesh.path = meshpath

# export toolbox
!jupyter nbconvert --to=python Py_f2recom_toolbox.ipynb

Processing years 1974 to 1978

Reading out of /albedo/scratch/user/simuel001/blanchard_newparams_20240822/
Storing graphs to /albedo/home/simuel001/master_thesis/master_thesis/py_f2recom_4phy/simuel001/blanchard_newparams_20240822/
Storing html to /albedo/home/simuel001/master_thesis/master_thesis/py_f2recom_4phy/simuel001/blanchard_newparams_20240822/ as simuel001/blanchard_newparams_20240822__2024_09_13_eco.html

Last ten years are 
[1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971
 1972 1973 1974 1975 1976 1977 1978]
/albedo/work/projects/p_bio/mesh/core2/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /albedo/work/projects/p_bio/mesh/core2/pickle_mesh_py3_fesom2
[NbConvertApp] Converting notebook Py_f2recom_toolbox.ipynb to python
[NbConvertApp] Writing 48334 bytes to Py_f2recom_toolbox.py


# 4. Ocean Productivity & ecosystem (NPP, Export and Chlorophyll a)<a class="anchor" id="PP"></a>

## 4.1 Maps<a class="anchor" id="PPMAPS"></a>

### 4.1.1 Global Ocean Chl a [OC-CCI] <a class="anchor" id="OCCCIPPMAP"></a>

In [ ]:
!jupyter nbconvert --to=python ChlGlobal.ipynb
from ChlGlobal import ChlGlobal
ChlGlobal(resultpath,savepath,mesh,matfileChlGloOCCCI,
          first_year_maps,last_year, savefig=savefig)

### 4.1.2 Southern ocean Chl a [Johnson et al. (2013)] <a class="anchor" id="JOHNPPMAP"></a>

In [ ]:
!jupyter nbconvert --to=python ChlSouthern.ipynb
from ChlSouthern import ChlSouthern
ChlSouthern(resultpath,savepath,mesh,ncfileChlSouthernJohnson,
                          first_year_maps,last_year,
                          savefig=savefig)

### 4.1.3 Arctic Ocean Chl a [Lewis et al. 2020 + OC-CCI]<a class="anchor" id="ARCCHLPMAP"></a>

In [ ]:
!jupyter nbconvert --to=python ChlArctic.ipynb
from ChlArctic import ChlArctic
ChlArctic(resultpath,savepath,mesh,ncfileChlArcLewis,
                          first_year_maps,last_year,savefig=savefig)

In [ ]:
!jupyter nbconvert --to=python ChlArctic.ipynb
from ChlArctic import ChlArctic
ChlArctic(resultpath,savepath,mesh,ncfileChlArcOCCCI,
                          first_year_maps,last_year,savefig=savefig)

### 4.1.4 Global Ocean NPP (cpbm & vgpn)<a class="anchor" id="NPPGLOBAL"></a>

In [ ]:
!jupyter nbconvert --to=python NPPGlobal.ipynb
from NPPGlobal import NPPGlobal

# analyze only last 10 years
# WARNING: here FESOM NPP is integrated and compared to Ocean Color NPP (surface)
NPPGlobal(resultpath, savepath, mesh, matfileNPPcpbm,
          first_year_maps, last_year, savefig=savefig)

In [ ]:
!jupyter nbconvert --to=python NPPGlobal.ipynb
from NPPGlobal import NPPGlobal

NPPGlobal(resultpath, savepath, mesh, matfileNPPvgpm,
                first_year_maps,last_year, savefig=savefig)

### 4.1.5 Arctic Ocean NPP [Lewis et al. 2020 & Globcolour]<a class="anchor" id="#NPPARCTICMAP"></a>

In [ ]:
!jupyter nbconvert --to=python NPPArctic.ipynb
from NPPArctic import NPPArctic

NPPArctic(resultpath,savepath,mesh,npfileNPPArcLewis,
                          first_year_maps,last_year,savefig=savefig)

In [ ]:
!jupyter nbconvert --to=python NPPArctic.ipynb
from NPPArctic import NPPArctic

NPPArctic(resultpath,savepath,mesh,npfileNPParcCMEMS,
                          first_year_maps,last_year,savefig=savefig)

### 4.1.6 Maredat Biomass maps <a class="anchor" id="MAREDAT"></a>

In [ ]:
!jupyter nbconvert --to=python Biomass_maps_all_FT.ipynb
from Biomass_maps_all_FT import Bio_map_all

Bio_map_all(resultpath,savepath,mesh,meshpath,first_year_maps,last_year)

#### Diatoms

In [ ]:
!jupyter nbconvert --to=python Biomass_Maredat.ipynb
from Biomass_Maredat import Bio_comp

Bio_comp(resultpath,savepath,mesh,meshpath,ncfileMaredatDia,'DiaC',
                          first_year_maps,last_year)

#### Coccolithophores

In [ ]:
!jupyter nbconvert --to=python Biomass_Maredat.ipynb
from Biomass_Maredat import Bio_comp

from pathlib import Path
cocco_path = Path(resultpath + '/CoccoC.fesom.'+str(years[0])+'.nc') # assuming that coccos were used for the entire simulation if they were used in the first year of simulation
if cocco_path.is_file():
    Bio_comp(resultpath,savepath,mesh,meshpath,ncfileMaredatCocco,'CoccoC',
                              first_year_maps,last_year)
else:
    print('Sorry, model was used without coccolithophores')

#### Phaeocystis

In [ ]:
!jupyter nbconvert --to=python Biomass_Maredat.ipynb
from Biomass_Maredat import Bio_comp

from pathlib import Path
phaeo_path = Path(resultpath + '/PhaeoC.fesom.'+str(years[0])+'.nc') # assuming that coccos were used for the entire simulation if they were used in the first year of simulation
if phaeo_path.is_file():
    Bio_comp(resultpath,savepath,mesh,meshpath,ncfileMaredatPhaeo,'PhaeoC',
                              first_year_maps,last_year)
else:
    print('Sorry, model was used without phaeocystis')

#### Microzooplankton

In [ ]:
!jupyter nbconvert --to=python Biomass_Maredat.ipynb
from Biomass_Maredat import Bio_comp

from pathlib import Path
micro_path = Path(resultpath + '/Zoo3C.fesom.'+str(years[0])+'.nc') # assuming that microzooplankton was used for the entire simulation if it was used in the first year of simulation
if micro_path.is_file():
    Bio_comp(resultpath,savepath,mesh,meshpath,ncfileMaredatMicro,'Zoo3C',
                              first_year_maps,last_year)
else:
    print('Sorry, model was used without microzooplankton (third zooplankton)')  

#### Mesozooplankton

In [ ]:
!jupyter nbconvert --to=python Biomass_Maredat.ipynb
from Biomass_Maredat import Bio_comp

Bio_comp(resultpath,savepath,mesh,meshpath,ncfileMaredatMeso,'HetC',
                          first_year_maps,last_year)

#### Mesozooplankton

In [ ]:
!jupyter nbconvert --to=python Biomass_Maredat.ipynb
from Biomass_Maredat import Bio_comp

from pathlib import Path
micro_path = Path(resultpath + '/Zoo2C.fesom.'+str(years[0])+'.nc') # assuming that macrozooplankton was used for the entire simulation if it was used in the first year of simulation
if micro_path.is_file():
    Bio_comp(resultpath,savepath,mesh,meshpath,ncfileMaredatMacro,'Zoo2C',
                              first_year_maps,last_year)
else:
    print('Sorry, model was used without macrozooplankton (second zooplankton / krill)')  

### 4.1.7 Limiting Factors <a class="anchor" id="LIMFACT"></a>

In [ ]:
!jupyter nbconvert --to=python LIMfactors.ipynb
!jupyter nbconvert --to=python load_interp_WOA.ipynb
from LIMfactors import LimFact
limfact = LimFact(resultpath, savepath, mesh, last_year, last_year, mapproj=mapproj,
                savefig=savefig, cmap = cmo.cm.thermal, verbose=verbose, frequency = 'yearly', plotting=True)
# Hatched areas are aras with weak limitation (all parameters >0.5)

## 4.2 Latitudinal distributions<a class="anchor" id="LATPP"></a>

In [ ]:
!jupyter nbconvert --to=python Chl_NPP_latitudinal.ipynb
!jupyter nbconvert --to=python ChlGlobal.ipynb
!jupyter nbconvert --to=python ChlSouthern.ipynb
!jupyter nbconvert --to=python NPPGlobal.ipynb
from Chl_NPP_latitudinal import Chl_NPP_lat_comp

Chl_NPP_lat_comp(resultpath,savepath,mesh,
                            matfileChlGloOCCCI,ncfileChlSouthernJohnson,matfileNPPcpbm,matfileNPPvgpm,
                            first_year_maps,last_year, savefig=savefig)

## 4.3 Time series<a class="anchor" id="TSPP"></a>

In [ ]:
# time-series of NPP and export production
!jupyter nbconvert --to=python NPP_timeseries.ipynb
from NPP_timeseries import NPP_timeseries
NPP_timeseries(resultpath,savepath,mesh,first_year_maps,last_year,savefig=savefig,mask="Global Ocean")
### export production --> gravitational sinking of detritus out of 100 m depth

In [ ]:
# Example for the Southern Ocean only
!jupyter nbconvert --to=python NPP_timeseries.ipynb
from NPP_timeseries import NPP_timeseries
NPP_timeseries(resultpath,savepath,mesh,first_year_maps,last_year,savefig=savefig,mask="Southern_Ocean_50S")

In [ ]:
!jupyter nbconvert --to=python NPP_timeseries.ipynb
from NPP_timeseries import NPP_timeseries
NPP_timeseries(resultpath,savepath,mesh,first_year,last_year,savefig=savefig,mask="Arctic_Basin")

## 4.4 Seasonal cycle<a class="anchor" id="SCyc"></a>




### 4.4.1 Chl-a<a class="anchor" id="SCycChla"></a>

In [ ]:
!jupyter nbconvert --to=python SeasonalCycle.ipynb
from SeasonalCycle import SeasonalCycle

SeasonalCycle(resultpath,savepath,mesh,first_year,last_year,type="Chl",savefig=False)

[NbConvertApp] Converting notebook SeasonalCycle.ipynb to python
[NbConvertApp] Writing 16889 bytes to SeasonalCycle.py


### 4.4.2 NPP<a class="anchor" id="SCycNPP"></a>

In [ ]:
!jupyter nbconvert --to=python SeasonalCycle.ipynb
from SeasonalCycle import SeasonalCycle

SeasonalCycle(resultpath,savepath,mesh,first_year,last_year,type="NPP",savefig=False)

# Save notebook as html

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')
%autosave
!jupyter nbconvert $nb_name --output-dir=$htmlpath --to html --no-input